In [56]:
import numpy as np
import pandas as pd
import re
from ast import literal_eval as le
import warnings as w
w.filterwarnings("ignore")

In [57]:
df = pd.read_csv("anime_data.csv")
pd.set_option("display.max_columns", None)

In [58]:
df

,anime_poster,anime_title,anime_overview,anime_mal_score,anime_views,anime_studio,anime_producer,anime_genres
0,https://img.flawlessfiles.com/_r/300x400/100/5...,"""Bungaku Shoujo"" Kyou no Oyatsu: Hatsukoi",\n Shor...,6.92,NaN,Production I.G,"['Lantis', 'Pony Canyon', 'Enterbrain', 'Kadok...","['Comedy', 'Fantasy', 'School']"
1,https://img.flawlessfiles.com/_r/300x400/100/e...,"""Bungaku Shoujo"" Memoire",\n Epis...,7.35,NaN,Production I.G,"['Lantis', 'Pony Canyon', 'Enterbrain', 'Kadok...","['Drama', 'Romance', 'School']"
2,https://img.flawlessfiles.com/_r/300x400/100/5...,"""Bungaku Shoujo"" Movie",\n The ...,7.43,NaN,Production I.G,"['Lantis', 'Pony Canyon', 'Enterbrain', 'Kadok...","['Mystery', 'Drama', 'Romance', 'School']"
3,https://img.flawlessfiles.com/_r/300x400/100/9...,My Star,\n In t...,9.33,NaN,Doga Kobo,"['Kadokawa', 'Shueisha']","['Drama', 'Music', 'Supernatural', 'Seinen']"
4,https://img.flawlessfiles.com/_r/300x400/100/e...,.hack//G.U. Returner,\n The ...,6.73,NaN,Bee Train,"['Bandai Visual', 'CyberConnect2', 'Bee Train']","['Adventure', 'Drama', 'Fantasy', 'Game', 'Mag..."
...,...,...,...,...,...,...,...,...
6077,https://img.flawlessfiles.com/_r/300x400/100/2...,[RAW] Yobarete Tobidete Akubi-chan,\n Koro...,5.94,NaN,Tatsunoko Production,['Tatsunoko Production'],"['Comedy', 'Fantasy', 'Kids']"
6078,https://img.flawlessfiles.com/_r/300x400/100/3...,[RAW] Youkai Ningen Bem,\n Thre...,6.02,NaN,Dai-Ichi Douga,['Dai-Ichi Douga'],"['Demons', 'Horror']"
6079,https://img.flawlessfiles.com/_r/300x400/100/2...,[RAW] Zoids Wild Zero,\n The ...,?,NaN,OLM,['OLM'],"['Action', 'Adventure', 'Comedy', 'Sci-Fi']"
6080,https://img.flawlessfiles.com/_r/300x400/100/5...,_Summer,\n “Is ...,5.7,NaN,Rikuentai,['Rikuentai'],"['Drama', 'Romance', 'Harem']"


In [59]:
df.anime_producer = df.anime_producer.replace("['NA']", df.anime_producer.mode()[0])
df.anime_mal_score = pd.to_numeric(df.anime_mal_score, errors="coerce")
df.anime_mal_score = df.anime_mal_score.replace(np.nan, df.anime_mal_score.median())
df.drop(columns='anime_views',axis=1,inplace=True)

In [60]:
df

,anime_poster,anime_title,anime_overview,anime_mal_score,anime_studio,anime_producer,anime_genres
0,https://img.flawlessfiles.com/_r/300x400/100/5...,"""Bungaku Shoujo"" Kyou no Oyatsu: Hatsukoi",\n Shor...,6.92,Production I.G,"['Lantis', 'Pony Canyon', 'Enterbrain', 'Kadok...","['Comedy', 'Fantasy', 'School']"
1,https://img.flawlessfiles.com/_r/300x400/100/e...,"""Bungaku Shoujo"" Memoire",\n Epis...,7.35,Production I.G,"['Lantis', 'Pony Canyon', 'Enterbrain', 'Kadok...","['Drama', 'Romance', 'School']"
2,https://img.flawlessfiles.com/_r/300x400/100/5...,"""Bungaku Shoujo"" Movie",\n The ...,7.43,Production I.G,"['Lantis', 'Pony Canyon', 'Enterbrain', 'Kadok...","['Mystery', 'Drama', 'Romance', 'School']"
3,https://img.flawlessfiles.com/_r/300x400/100/9...,My Star,\n In t...,9.33,Doga Kobo,"['Kadokawa', 'Shueisha']","['Drama', 'Music', 'Supernatural', 'Seinen']"
4,https://img.flawlessfiles.com/_r/300x400/100/e...,.hack//G.U. Returner,\n The ...,6.73,Bee Train,"['Bandai Visual', 'CyberConnect2', 'Bee Train']","['Adventure', 'Drama', 'Fantasy', 'Game', 'Mag..."
...,...,...,...,...,...,...,...
6077,https://img.flawlessfiles.com/_r/300x400/100/2...,[RAW] Yobarete Tobidete Akubi-chan,\n Koro...,5.94,Tatsunoko Production,['Tatsunoko Production'],"['Comedy', 'Fantasy', 'Kids']"
6078,https://img.flawlessfiles.com/_r/300x400/100/3...,[RAW] Youkai Ningen Bem,\n Thre...,6.02,Dai-Ichi Douga,['Dai-Ichi Douga'],"['Demons', 'Horror']"
6079,https://img.flawlessfiles.com/_r/300x400/100/2...,[RAW] Zoids Wild Zero,\n The ...,7.09,OLM,['OLM'],"['Action', 'Adventure', 'Comedy', 'Sci-Fi']"
6080,https://img.flawlessfiles.com/_r/300x400/100/5...,_Summer,\n “Is ...,5.70,Rikuentai,['Rikuentai'],"['Drama', 'Romance', 'Harem']"


In [61]:
df.isna().sum()

anime_poster         0
anime_title          0
anime_overview       0
anime_mal_score      0
anime_studio       504
anime_producer       0
anime_genres         0
dtype: int64

In [62]:
df.dropna(inplace=True)

In [63]:
df.anime_genres = df.anime_genres.apply(le)
df.anime_producer = df.anime_producer.apply(le)
df.anime_genres = df.anime_genres.apply(lambda x: [i.replace(" ", "") for i in x])
df.anime_studio = df.anime_studio.apply(lambda x: x.replace(" ", ""))
df.anime_studio = df.anime_studio.apply(lambda x: x.split())

In [64]:
#Fetch wordcount for each abstract

df['word_count'] = df['anime_overview'].apply(lambda x: len(str(x).split(" ")))
df[['anime_overview','word_count']].head()

,anime_overview,word_count
0,\n Shor...,90
1,\n Epis...,88
2,\n The ...,187
3,\n In t...,218
4,\n The ...,146


In [65]:
##Descriptive statistics of word counts

df.word_count.describe()

count    5578.000000
mean      173.353532
std        65.518671
min         1.000000
25%       125.000000
50%       172.000000
75%       224.000000
max       533.000000
Name: word_count, dtype: float64

In [66]:
#Identify common words
freq = pd.Series(' '.join(df['anime_overview']).split()).value_counts()[:20]
freq

the      34282
to       18699
of       16330
a        16001
and      15839
is        8155
in        7977
his       6656
with      5410
her       5013
by        4986
that      4429
as        4399
their     4002
he        3839
for       3762
on        2942
an        2928
who       2873
from      2785
Name: count, dtype: int64

In [67]:
# Identify uncommon words
freq1 = pd.Series(" ".join(df["anime_overview"]).split()).value_counts()[-20:]
freq1

Hasegawa's        1
Fist.             1
chinese           1
Caproni.          1
A6M               1
Sukunai.          1
Tomodachi         1
Satomi;           1
malaise           1
comedy!!          1
II.For            1
fiber             1
wine),            1
onward⁠—even      1
change...         1
dream?[Written    1
(1600–1868),      1
live—the          1
(rice             1
Kakana,           1
Name: count, dtype: int64

# Text Preprocessing

In [68]:
# Removing the last part from the overview which is [Written by MAL Rewrite] 

df['anime_overview'] =  df['anime_overview'].apply(lambda x: x.split(r"[Written by MAL Rewrite]")[0])

# Taking Only Words
df['anime_overview'] = df['anime_overview'].apply(lambda x: " ".join(re.findall(r'[a-zA-Z]+', x)).lower())

# Removing all s and t from the overview which are not in the form of words

df['anime_overview'] = df['anime_overview'].apply(lambda x: re.sub(r'\bs\b|\bt\b','',x))


In [69]:
df

,anime_poster,anime_title,anime_overview,anime_mal_score,anime_studio,anime_producer,anime_genres,word_count
0,https://img.flawlessfiles.com/_r/300x400/100/5...,"""Bungaku Shoujo"" Kyou no Oyatsu: Hatsukoi",short episode bundled with the limited edition...,6.92,[ProductionI.G],"[Lantis, Pony Canyon, Enterbrain, Kadokawa Con...","[Comedy, Fantasy, School]",90
1,https://img.flawlessfiles.com/_r/300x400/100/e...,"""Bungaku Shoujo"" Memoire",episodes which depict the background stories o...,7.35,[ProductionI.G],"[Lantis, Pony Canyon, Enterbrain, Kadokawa Con...","[Drama, Romance, School]",88
2,https://img.flawlessfiles.com/_r/300x400/100/5...,"""Bungaku Shoujo"" Movie",the protagonist of the story konoha inoue is a...,7.43,[ProductionI.G],"[Lantis, Pony Canyon, Enterbrain, Kadokawa Con...","[Mystery, Drama, Romance, School]",187
3,https://img.flawlessfiles.com/_r/300x400/100/9...,My Star,in the entertainment world celebrities often s...,9.33,[DogaKobo],"[Kadokawa, Shueisha]","[Drama, Music, Supernatural, Seinen]",218
4,https://img.flawlessfiles.com/_r/300x400/100/e...,.hack//G.U. Returner,the characters from previous hack g u games an...,6.73,[BeeTrain],"[Bandai Visual, CyberConnect2, Bee Train]","[Adventure, Drama, Fantasy, Game, Magic, Sci-Fi]",146
...,...,...,...,...,...,...,...,...
6077,https://img.flawlessfiles.com/_r/300x400/100/2...,[RAW] Yobarete Tobidete Akubi-chan,koron nemuta is a girl who yawns when embrasse...,5.94,[TatsunokoProduction],[Tatsunoko Production],"[Comedy, Fantasy, Kids]",120
6078,https://img.flawlessfiles.com/_r/300x400/100/3...,[RAW] Youkai Ningen Bem,three monsters bem who looks like a gangster b...,6.02,[Dai-IchiDouga],[Dai-Ichi Douga],"[Demons, Horror]",131
6079,https://img.flawlessfiles.com/_r/300x400/100/2...,[RAW] Zoids Wild Zero,the story revolves around sally search for he...,7.09,[OLM],[OLM],"[Action, Adventure, Comedy, Sci-Fi]",190
6080,https://img.flawlessfiles.com/_r/300x400/100/5...,_Summer,is there any girl you love by this word his fr...,5.70,[Rikuentai],[Rikuentai],"[Drama, Romance, Harem]",173


In [70]:
# Dropping the index column and resetting the index to new values
df.reset_index(drop=True, inplace=True)
df.reset_index(inplace=True)

In [ ]:
# Changing the column names

df.rename(
    columns={
        "index": "anime_id",
        "anime_urls": "urls",
        "anime_overview": "overview",
        "anime_genres": "genres",
        "anime_producer": "producer",
        "anime_studio": "studio",
        "anime_mal_score": "score",
        "anime_poster": "poster",
        "anime_title": "title",
        "anime_views": "views",
    },
    inplace=True,
)

In [ ]:
df.to_csv("anime_data_cleaned.csv", index=False)